In [1]:
import sympy as sp
import numpy as np
from sympy import Matrix

In [3]:
def max_algebra_multiply_symbolic(A, B, sym_val_dict):
    rows_A, cols_A = A.shape
    rows_B, cols_B = B.shape
    if cols_A != rows_B:
        raise ValueError("Matrices are not compatible for multiplication")
    C = Matrix.zeros(rows_A, cols_B)
    
    for i in range(rows_A):
        for j in range(cols_B):
            max_val = -float('inf')
            max_expression = None
            for k in range(cols_A):
                cur_val = (A[i,k] * B[k,j]).subs(sym_val_dict)
                    
                if cur_val > max_val:
                    max_val = cur_val
                    max_expression = A[i,k] * B[k,j]

            C[i,j] = max_expression

    return C

In [4]:
def max_matrix_power_symbolic(C, k, sym_val_dict):
    if k == 0:
        return Matrix.eye(C.shape[0])
    elif k == 1:
        return C
    else:
        result = C
        for _ in range(k - 1):
            result = max_algebra_multiply_symbolic(result, C, sym_val_dict)
        return result

In [5]:
def max_algebra_addition_symbolic(matrices, sym_val_dict): 
    
    shape = matrices[0].shape
    result = Matrix.zeros(shape[0], shape[1])
    
    for i in range(shape[0]):
        for j in range(shape[1]):
            max_value = -np.inf
            for matrix in matrices:
                matrix_ij_val = matrix[i, j].subs(sym_val_dict)
                if matrix_ij_val > max_value:
                    max_value = matrix_ij_val
                    result[i, j] = matrix[i, j]
    
    return result

In [5]:
def trace_symbolic(C, sym_val_dict):
    diagonal_elements = Matrix.diagonal(C)
    max_value = -np.inf
    for el in diagonal_elements:
        el_num = el.subs(sym_val_dict)
        if el_num > max_value:
            max_value = el_num
            max_element = el
    return max_element

In [6]:
def spectral_radius_symbolic(C, sym_val_dict):
    k = C.shape[0]
    temp = C
    max_value = -np.inf
    max_expression = None
    for i in range(1, k + 1):
        tr = trace_symbolic(temp, sym_val_dict) ** sp.Rational(1, i)
        num_tr = tr.subs(sym_val_dict)
        temp = max_algebra_multiply_symbolic(temp, C, sym_val_dict)
        if num_tr > max_value:
            max_value = num_tr
            max_expression = tr
    return max_expression

In [7]:
def klini_symbolic(C, sym_val_dict):
    r = list(sym_val_dict.keys())[-1]
    k = C.shape[0]
    tmp = Matrix.eye(k)
    matrices = [Matrix.eye(k)]
    for i in range(1, k):
        matrices.append(r**(-i)*max_algebra_multiply_symbolic(tmp, C, sym_val_dict))
        tmp = max_algebra_multiply_symbolic(tmp, C, sym_val_dict)
    return max_algebra_addition_symbolic(matrices, sym_val_dict)

In [ ]:
# Выше - часть с лог-чебышёвской аппроксимацией

In [8]:
def saati_method(A, epsilon = 1e-9, max_iterations=1000):

        n = A.shape[0]

        x = sp.Matrix([1 for _ in range(n)])
        x_prev = np.array(x.tolist(), dtype=float) # Convert to numpy before using it
        x_curr = x_prev / np.linalg.norm(x_prev)  # Normalize the initial vector

        for i in range(max_iterations):
            norm_prev = np.linalg.norm(x_prev)
            x_prev = x_curr.copy()
            x_curr = np.array((A * sp.Matrix(x_prev)).tolist(), dtype = float) # perform the operation using sympy matrix, convert to numpy array
            norm_curr = np.sum(x_curr)
            x_curr = x_curr / norm_curr  # Normalize at each iteration
            lambd = norm_curr / norm_prev
            if np.linalg.norm(x_curr - x_prev) < epsilon:
                return [x_curr, lambd, i]  # Return flattened eigenvector

        return None  # Did not converge

In [9]:
def apply_saati_method(matrices, epsilon, max_iterations=1000):
    """Applies the iterative function to multiple matrices using NumPy efficiently."""
    results = []
    for matrix in matrices:
        result = saati_method(matrix, epsilon, max_iterations)
        results.append(result)
    return results #Return as a NumPy array

In [10]:
C = sp.Matrix([[1, 3, 5, 7, 2],
              [sp.Rational(1,3), 1, 3, 3, sp.Rational(1,5)],
              [sp.Rational(1,5), sp.Rational(1,3), 1, 2, sp.Rational(1,4)],
              [sp.Rational(1,7), sp.Rational(1,3), sp.Rational(1,2), 1, sp.Rational(1,3)],
              [sp.Rational(1,2), 5, 4, 3, 1]])

In [11]:
def cumprod_row(matrix):
    rows, cols = matrix.shape
    return Matrix([[sp.prod(matrix[r, :c+1]) for c in range(cols)] for r in range(rows)])

In [20]:
def geom_means(matrix):
    rows, cols = matrix.shape
    result = []

    for r in range(rows):
      row_prod = sp.prod(matrix[r, :])  # Calculate product of elements in current row
      nth_root = sp.root(row_prod, cols) # Take the n-th root
      result.append(float(nth_root))
    
    return result

In [14]:
C = sp.Matrix([[1, 3, 5, 7, 2],
              [sp.Rational(1,3), 1, 3, 3, sp.Rational(1,5)],
              [sp.Rational(1,5), sp.Rational(1,3), 1, 2, sp.Rational(1,4)],
              [sp.Rational(1,7), sp.Rational(1,3), sp.Rational(1,2), 1, sp.Rational(1,3)],
              [sp.Rational(1,2), 5, 4, 3, 1]])

A_1 = sp.Matrix([[1, 2, 3, 7, 4],
              [sp.Rational(1,2), 1, 4, sp.Rational(1,4), sp.Rational(1,3)],
              [sp.Rational(1,3), sp.Rational(1,4), 1, sp.Rational(1,5), sp.Rational(1,2)],
              [sp.Rational(1,7), sp.Rational(1,4), 5, 1, sp.Rational(1,3)],
              [sp.Rational(1,4), 3, 2, 3, 1]])

A_2 = sp.Matrix([[1, 2, 3, 6, 4],
              [sp.Rational(1,3), 1, 6, 6, 6],
              [sp.Rational(1,5), sp.Rational(1,6), 1, 3, 4],
              [sp.Rational(1,6), sp.Rational(1,6), sp.Rational(1,3), 1, 3],
              [sp.Rational(1,4), sp.Rational(1,6), sp.Rational(1,4), sp.Rational(1,3), 1]])

A_3 = sp.Matrix([[1, 2, 3, sp.Rational(1,2), sp.Rational(1,5)],
              [sp.Rational(1,2), 1, 2, sp.Rational(1,4), sp.Rational(1,4)],
              [sp.Rational(1,3), sp.Rational(1,2), 1, sp.Rational(1,6), sp.Rational(1,6)],
              [2, 4, 6, 1, sp.Rational(1,2)],
              [5, 4, 6, 2, 1]])

A_4 = sp.Matrix([[1, 3, 5, 7, 9],
              [sp.Rational(1,3), 1, 2, 5, 5],
              [sp.Rational(1,5), sp.Rational(1,3), 1, 3, 3],
              [sp.Rational(1,7), sp.Rational(1,5), sp.Rational(1,3), 1, 2],
              [sp.Rational(1,9), sp.Rational(1,5), sp.Rational(1,3), sp.Rational(1,2), 1]])

A_5 = sp.Matrix([[1, 2, 3, 2, 3],
              [sp.Rational(1,2), 1, 3, 4, 5],
              [sp.Rational(1,3), sp.Rational(1,3), 1, 2, 2],
              [sp.Rational(1,3), sp.Rational(1,4), sp.Rational(1,2), 1, 3],
              [sp.Rational(1,2), sp.Rational(1,5), sp.Rational(1,2), sp.Rational(1,3), 1]])
res = [C, A_1, A_2, A_3, A_4, A_5]

In [15]:
res_means = list(map(geom_means, res))
res_means
saati = list(map(saati_method, res))
saati
saati = [sublist[0] for sublist in saati]

In [16]:
def weighted_sum_of_vectors(list_of_arrays):
    c = list_of_arrays[0]
    vectors = list_of_arrays[1:]
    c = np.array(c)  # Ensure c is a NumPy array
    vectors = [np.array(v) for v in vectors] # Ensure all vectors are numpy arrays
    weighted_sum = np.zeros_like(vectors[0], dtype = float)  # Initialize with zeros (with float type to prevent int overflows
    for i, vector in enumerate(vectors):
       weighted_sum += c[i] * vector #Multiply vector by the coefficient and sum with the previous vectors
    return [weighted_sum, weighted_sum/max(weighted_sum)]

In [17]:
weighted_sum_of_vectors(res_means)

[array([16.55497629,  9.01611159,  4.0649428 ,  4.52924   ,  6.73752204]),
 array([1.        , 0.5446164 , 0.24554205, 0.27358783, 0.40697866])]

In [18]:
def KrivulinFull(C_then_A_i_matrices):
    saati = list(map(saati_method, C_then_A_i_matrices))
    saati = [sublist[0] for sublist in saati]
    saati[0] = saati[0]/np.sum(saati[0])
    # return weighted_sum_of_vectors(saati)
    weighted_sum_of_vectors(saati)

    geomeans = list(map(geom_means, C_then_A_i_matrices))

In [19]:
KrivulinFull(res)